In [1]:
import boto3

# Let's use Amazon S3
s3c = boto3.client('s3')
s3r = boto3.resource('s3')

In [2]:
buckets = []

# Print out bucket names
for bucket in s3r.buckets.all():
    buckets.append(bucket)

In [3]:
for i in buckets[1].objects.iterator():
    pass
    #print(i)

In [4]:
documents = s3c.list_objects(Bucket = buckets[1].name)['Contents']

In [5]:
textract_c = boto3.client('textract')

In [6]:
documents_ls = [d['Key'] for d in documents]
#documents_ls

In [7]:
response_ls = []
for d in documents_ls:
    response_ls.append(textract_c.start_document_analysis(DocumentLocation={'S3Object':{'Bucket':buckets[1].name,'Name':d}},FeatureTypes = ['TABLES']))

In [8]:
jobId_ls = [r['JobId'] for r in response_ls]

In [13]:
result_ls = []
for j in jobId_ls:
    result_ls.append(textract_c.get_document_analysis(JobId=j))

In [15]:
import pandas as pd

jobStatus = [r['JobStatus'] for r in result_ls]
jobStatus_df = pd.DataSeries(jobStatus)


AttributeError: module 'pandas' has no attribute 'DataSeries'

In [20]:
def creatRaw(result):

    raw_ls=[]
    raw_txt=''
    for t in result['Blocks']:
        try:
            if t['Text'][:4]=='Page':
                pass
            else:
                raw_ls.append( t['Text'])
                raw_txt += t['Text']+' '
        except:
            pass

    return raw_txt,raw_ls

In [27]:
import json

In [29]:
file_head = 'city of san jose minutes_'
file_surfix = '.json'
i = 0

for r in result_ls:
    raw_txt,raw_ls = creatRaw(result_ls[i])
    with open(file_head+str(i+1)+file_surfix,'w') as f:
        json.dump(result_ls[i],f)
        f.close()
    i+=1
    